In [1]:
from rsplib import RSPEngine, RSPPublisher, Stream, rdf_table,load_graph, accessURL

In [2]:
jasper = RSPEngine("http://jasper:8181/jasper")

In [9]:
jasper.streams()[2].endpoints()[0].call()

# If you did not create the yellow color stream before go back to Ex4

In [3]:
qid  = 'reasoning2'
tbox = 'https://raw.githubusercontent.com/riccardotommasini/colorwave/master/colors.owl'
frmt = 'JSON'
body = '''
    
    SELECT (COUNT(?p) AS ?cntPrimary) (COUNT(?d) AS ?cntDerived) (?cntPrimary > ?cntDerived AS ?morePrimaryThanDerived)
    FROM NAMED WINDOW <rw> ON <http://redstream:1255/red> [RANGE PT15S STEP PT5S]
    FROM NAMED WINDOW <gw> ON <http://greenstream:2255/green> [RANGE PT15S STEP PT5S]
    FROM NAMED WINDOW <bw> ON <http://bluestream:3255/blue> [RANGE PT15S STEP PT5S]
    FROM NAMED WINDOW <yw> ON <http://jasper:8181/jasper/streams/yellow> [RANGE PT15S STEP PT5S]
    WHERE {
        { WINDOW ?pw { ?p a <http://www.streamreasoning.org/ontologies/2018/9/colors#Primary> . }}
        UNION
        { WINDOW ?dw { ?d a <http://www.streamreasoning.org/ontologies/2018/9/colors#Derived>. }}
    }

    '''

In [4]:
jasper.create(qid, body, tbox, frmt)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
r = jasper.expose(qid, 'HTTP', retention=5)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
jasper.streams()

reasoning2
